# Group existing generators within regions and create new resources

This notebook shows how to use the PowerGenome API to cluster existing resources and create new-build resource options by model region. You'll need a settings file, such as the `test_settings.yml` provided in the `example_system` folder, with the following parameters.

For both existing and new resource:
- model_regions
- region_aggregations
- model_year
- target_usd_year
- atb_usd_year
- startup_fuel_use
- startup_vom_costs_mw
- startup_vom_costs_usd_year
- startup_costs_type
- startup_costs_per_cold_start_mw
- startup_costs_per_cold_start_usd_year
- existing_startup_costs_tech_map
- new_build_startup_costs

Specific to existing resource:
- num_clusters
- retirement_ages
- atb_existing_year
- existing_om_muiltiplier
- eia_atb_tech_map
- proposed_status_included
- proposed_gen_heat_rates
- proposed_min_load

Specific to new-build resources:
- atb_cost_case
- atb_financial_case
- atb_cap_recovery_years
- atb_new_gen
- cost_multiplier_region_map
- cost_multiplier_technology_map

To calculate fuel costs for each resource:
- aeo_fuel_region_map
- eia_series_region_names
- eia_series_fuel_names
- eia_aeo_year
- eia_series_scenario_names
- aeo_fuel_scenarios
- aeo_fuel_usd_year
- tech_fuel_map
- fuel_emission_factors
- carbon_tax (optional)

And if CCS resources are included:
- ccs_fuel_map
- ccs_capture_rate
- ccs_disposal_cost

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import warnings

warnings.simplefilter("ignore")

In [21]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
from powergenome.generators import GeneratorClusters
from powergenome.GenX import reduce_time_domain
from powergenome.load_profiles import make_final_load_curves
from powergenome.params import DATA_PATHS
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
)
from powergenome.external_data import (
    make_demand_response_profiles,
    make_generator_variability,
)

pd.options.display.max_columns = 200

## Import settings
This assumes that the settings file is set up for multiple scenarios/planning periods. If you are using a settings file with only a single scenario/planning period, remove or comment out the line with `build_scenario_settings`.

In [14]:
pudl_engine, pudl_out = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd.parent / "example_system" / "test_settings.yml"
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

## Initialize a `GeneratorClusters` object

`GeneratorClusters` is how existing generators are clustered, and it provides a convinience method for creating new-build resources (`GeneratorClusters.create_new_generators`).

In [7]:
gc = GeneratorClusters(pudl_engine, pudl_out, scenario_settings[2030]["p1"])

961.5000000000002  MW without lat/lon


### Existing generators

Lets look at how existing generators have been clustered in the different regions. The raw output from `create_region_technology_clusters` has lots of extra columns that might not be needed. Some of them include:

- `resource` is a snake case version of `technology`.
- `unmodified_cap_size` is the average size of generating units. `Cap_size` is `unmodified_cap_size` multiplied by the capacity factor for technologies that are derated by their CF.
- `Existing_Cap_MW` is `Cap_size` multiplied by `num_units`.
- `heat_rate_mmbtu_mwh_iqr` and `heat_rate_mmbtu_mwh_std` are measurements of how widely the heate rate varies within a cluster. If these values are large compared to `Heat_rate_MMBTU_per_MWh` then you might consider increasing the number of clusters for that technology/region.
- `Fuel` is a combination of the region (assigned by the settings parameter `fuel_region_map`), the AEO scenario (assigned by the settings parameter `aeo_fuel_scenarios`), and the fuel type for that generator resource (assigned by the settings parameters `tech_fuel_map` and `ccs_fuel_map`). Only fuels in AEO are included, which means that biomass/hydrogen/RNG, etc are not options.

The various "model tags" that are included in the settings file are unique to GenX. If you find them useful, they can be used to assign various values of any data type to generators in a column name of your choosing. If you don't want to use them, they can be ignored or removed via `settings.pop(<name>)`.

In [ ]:
# gc.settings.pop("model_tag_names")
# gc.settings.pop("model_tag_values")
# gc.settings.pop("regional_tag_values")

In [6]:
existing_gen = gc.create_region_technology_clusters()

Creating gdf
['Solar Photovoltaic', 'Onshore Wind Turbine', 'Batteries', 'Biomass', 'Conventional Hydroelectric', 'Natural Gas Fired Combined Cycle', 'All Other', 'Natural Gas Fired Combustion Turbine', 'Other Natural Gas']


No model tag values found for DR ('NoneType' object has no attribute 'items')


In [81]:
existing_gen

,region,technology,cluster,index,Cap_size,minimum_load_mw,Heat_rate_MMBTU_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,Min_power,num_units,capacity_factor,unmodified_cap_size,Existing_Cap_MW,unmodified_existing_cap_mw,Fixed_OM_cost_per_MWyr,Var_OM_cost_per_MWh,Start_fuel_MMBTU_per_MW,Fuel,Start_cost_per_MW,THERM,DISP,NDISP,STOR,DR,HYDRO,Commit,RPS,CES,New_Build,Hydro_level,Resource,profile
0,CA_N,Biomass,1,0,0.91,0.973,17.202,1.484,4.645,0.374,81,0.350,2.602,73.71,210.762,142342,7.1,0.0,None,0.0,1,0,1,0,0,0,1,1,1,-1,0.0,biomass,None
1,CA_N,Conventional Hydroelectric,1,1,32.01,1.999,9.118,0.000,0.006,0.062,250,0.310,32.014,8002.50,8003.500,68272,0.0,0.0,None,0.0,0,0,0,0,0,1,0,0,1,-1,0.5,conventional_hydroelectric,"[0.21636393788244979, 0.21612066334138946, 0.2..."
2,CA_N,Geothermal,1,2,27.95,14.878,9.118,0.000,0.000,0.389,23,0.730,38.278,642.85,880.394,135226,0.0,0.0,None,0.0,1,0,1,0,0,0,0,1,1,-1,0.0,geothermal,None
3,CA_N,Hydroelectric Pumped Storage,1,3,89.55,23.850,0.000,0.000,0.000,0.266,20,-0.022,89.550,1791.00,1791.000,68272,0.0,0.0,None,0.0,0,0,0,1,0,0,0,0,0,0,0.0,hydroelectric_pumped_storage,None
4,CA_N,Natural Gas Fired Combined Cycle,1,4,309.16,148.158,7.476,1.416,1.473,0.479,33,0.374,309.161,10202.28,10202.313,13080,3.9,2.0,pacific_reference_naturalgas,87.0,1,0,0,0,0,0,1,0,0,1,0.0,natural_gas_fired_combined_cycle,None
5,CA_N,Natural Gas Fired Combustion Turbine,1,5,57.27,29.941,10.691,2.225,1.993,0.523,59,0.149,57.266,3378.93,3378.694,8090,11.8,3.5,pacific_reference_naturalgas,113.0,1,0,0,0,0,0,1,0,0,1,0.0,natural_gas_fired_combustion_turbine,None
6,CA_N,Onshore Wind Turbine,1,6,47.66,4.307,9.118,0.000,0.024,0.090,28,0.289,47.657,1327.60,1334.396,43205,0.0,0.0,None,0.0,0,1,0,0,0,0,0,1,1,0,0.0,onshore_wind_turbine,"[0.0004, 0.0293, 0.0617, 0.0811, 0.0819, 0.015..."
7,CA_N,Solar Photovoltaic,1,7,17.27,0.000,9.134,0.000,0.022,0.000,190,0.217,17.275,2787.10,3282.250,17943,0.0,0.0,None,0.0,0,1,0,0,0,0,0,1,1,0,0.0,solar_photovoltaic,"[0.2899, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
8,CA_S,Biomass,1,8,1.70,2.190,12.443,1.510,2.760,0.584,62,0.453,3.753,105.40,232.686,102962,5.1,0.0,None,0.0,1,0,1,0,0,0,1,1,1,-1,0.0,biomass,None
9,CA_S,Conventional Hydroelectric,1,9,11.35,1.510,9.004,0.000,1.859,0.133,163,0.330,11.353,1850.05,1850.539,68272,0.0,0.0,None,0.0,0,0,0,0,0,1,0,0,1,-1,0.5,conventional_hydroelectric,"[0.10526978282469837, 0.10516175443577006, 0.1..."


In [16]:
existing_gen

,region,technology,cluster,index,Cap_size,minimum_load_mw,Heat_rate_MMBTU_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,Min_power,num_units,capacity_factor,unmodified_cap_size,Existing_Cap_MW,unmodified_existing_cap_mw,Fixed_OM_cost_per_MWyr,Var_OM_cost_per_MWh,Start_fuel_MMBTU_per_MW,Fuel,Start_cost_per_MW,THERM,DISP,NDISP,STOR,DR,HYDRO,Commit,RPS,CES,New_Build,Hydro_level,Resource,profile
0,CA_N,Biomass,1,0,0.91,0.973,17.202,1.484,4.645,0.374,81,0.350,2.602,73.71,210.762,142342,7.1,0.0,None,0.0,1,0,1,0,0,0,1,1,1,-1,0.0,biomass,None
1,CA_N,Conventional Hydroelectric,1,1,62.17,3.740,9.118,0.000,0.009,0.060,117,0.308,62.171,7273.89,7274.007,68272,0.0,0.0,None,0.0,0,0,0,0,0,1,0,0,1,-1,0.5,conventional_hydroelectric,"[0.21636393788244979, 0.21612066334138946, 0.2..."
2,CA_N,Geothermal,1,2,27.95,14.878,9.118,0.000,0.000,0.389,23,0.730,38.278,642.85,880.394,135226,0.0,0.0,None,0.0,1,0,1,0,0,0,0,1,1,-1,0.0,geothermal,None
3,CA_N,Hydroelectric Pumped Storage,1,3,89.55,23.850,0.000,0.000,0.000,0.266,20,-0.022,89.550,1791.00,1791.000,68272,0.0,0.0,None,0.0,0,0,0,1,0,0,0,0,0,0,0.0,hydroelectric_pumped_storage,None
4,CA_N,Natural Gas Fired Combined Cycle,1,4,309.16,148.158,7.476,1.416,1.473,0.479,33,0.374,309.161,10202.28,10202.313,13080,3.9,2.0,pacific_reference_naturalgas,87.0,1,0,0,0,0,0,1,0,0,1,0.0,natural_gas_fired_combined_cycle,None
5,CA_N,Natural Gas Fired Combustion Turbine,1,5,57.27,29.941,10.691,2.225,1.993,0.523,59,0.149,57.266,3378.93,3378.694,8090,11.8,3.5,pacific_reference_naturalgas,113.0,1,0,0,0,0,0,1,0,0,1,0.0,natural_gas_fired_combustion_turbine,None
6,CA_N,Onshore Wind Turbine,1,6,47.66,4.307,9.118,0.000,0.024,0.090,28,0.289,47.657,1327.60,1334.396,43205,0.0,0.0,None,0.0,0,1,0,0,0,0,0,1,1,0,0.0,onshore_wind_turbine,"[0.0004, 0.0293, 0.0617, 0.0811, 0.0819, 0.015..."
7,CA_N,Small Hydroelectric,1,7,1.81,0.468,9.118,0.000,0.002,0.085,133,0.330,5.484,240.73,729.372,97272,0.0,0.0,None,0.0,0,0,1,0,0,0,0,1,1,-1,0.0,small_hydroelectric,"[0.21636393788244979, 0.21612066334138946, 0.2..."
8,CA_N,Solar Photovoltaic,1,8,17.27,0.000,9.134,0.000,0.022,0.000,190,0.217,17.275,2787.10,3282.250,17943,0.0,0.0,None,0.0,0,1,0,0,0,0,0,1,1,0,0.0,solar_photovoltaic,"[0.2899, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
9,CA_S,Biomass,1,9,1.70,2.190,12.443,1.510,2.760,0.584,62,0.453,3.753,105.40,232.686,102962,5.1,0.0,None,0.0,1,0,1,0,0,0,1,1,1,-1,0.0,biomass,None


In [17]:
make_generator_variability(existing_gen)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.0,0.216364,1.0,1.0,1.0,1.0,0.0004,0.216364,0.2899,1.0,0.105270,1.0,1.0,1.0,1.0,0.016339,0.105270,0.282178
1,1.0,0.216121,1.0,1.0,1.0,1.0,0.0293,0.216121,0.0000,1.0,0.105162,1.0,1.0,1.0,1.0,0.112207,0.105162,0.000000
2,1.0,0.215877,1.0,1.0,1.0,1.0,0.0617,0.215877,0.0000,1.0,0.105108,1.0,1.0,1.0,1.0,0.302212,0.105108,0.000000
3,1.0,0.215634,1.0,1.0,1.0,1.0,0.0811,0.215634,0.0000,1.0,0.105053,1.0,1.0,1.0,1.0,0.506039,0.105053,0.000000
4,1.0,0.215384,1.0,1.0,1.0,1.0,0.0819,0.215384,0.0000,1.0,0.104999,1.0,1.0,1.0,1.0,0.476053,0.104999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,0.217673,1.0,1.0,1.0,1.0,0.0204,0.217673,0.6897,1.0,0.105595,1.0,1.0,1.0,1.0,0.094797,0.105595,0.689717
8756,1.0,0.217430,1.0,1.0,1.0,1.0,0.0000,0.217430,0.7108,1.0,0.105541,1.0,1.0,1.0,1.0,0.076850,0.105541,0.731451
8757,1.0,0.217187,1.0,1.0,1.0,1.0,0.0000,0.217187,0.6974,1.0,0.105433,1.0,1.0,1.0,1.0,0.021176,0.105433,0.730564
8758,1.0,0.216943,1.0,1.0,1.0,1.0,0.0000,0.216943,0.6187,1.0,0.105377,1.0,1.0,1.0,1.0,0.003680,0.105377,0.640714


### New generators

The raw output from `create_new_generators` has many more columns than existing generators. Several of these - such as `basis_year`, `capex`, `capex_mwh`, `cap_recovery_years`, `waccnomtech`, `regional_cost_multiplier`, and `interconnect_annuity` - are used to calculate the final `Inv_cost_per_MWyr` and `Inv_cost_per_MWhyr`. `lcoe` is pre-calculated using 2030 mid-range ATB costs, and is not specific to the model year for each run. The underlying data are retained here so they can be easily reviewed, but only keep the columns that you want/need.

The column `variability` has array values with the annual generation profiles for a resource. If these profiles represent 2012 they have 8784 hourly values.

The column `Max_Cap_MW` has a value of -1 if there is no limit on the capacity. 

If the user has included demand response profiles as an external file (using the `demand_response_fn` parameter in settings and the associated `demand_response` and `demand_response_resources` parameters), demand response resources will be included 

In [53]:
new_gen = gc.create_new_generators()

Selected technology landbasedwind capacity in region CA_N less than minimum (8424.4314 < 25000 MW)
Selected technology landbasedwind capacity in region CA_S less than minimum (23506.7075 < 45000 MW)
No model tag values found for DR ('NoneType' object has no attribute 'items')
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.


In [13]:
new_gen.columns

Index(['technology', 'basis_year', 'Fixed_OM_cost_per_MWyr',
       'Fixed_OM_cost_per_MWhyr', 'Var_OM_cost_per_MWh', 'capex', 'capex_mwh',
       'Inv_cost_per_MWyr', 'Inv_cost_per_MWhyr', 'Heat_rate_MMBTU_per_MWh',
       'Cap_size', 'cap_recovery_years', 'waccnomtech', 'Max_Cap_MW', 'region',
       'regional_cost_multiplier', 'ids', 'area', 'm_popden', 'metro_id',
       'ipm_region', 'site_substation_spur_miles', 'substation_metro_tx_miles',
       'site_metro_spur_miles', 'interconnect_annuity', 'lcoe', 'spur_miles',
       'tx_miles', 'variability', 'offshore_spur_miles', 'turbine_type',
       'pref_site', 'Start_fuel_MMBTU_per_MW', 'Fuel', 'Start_cost_per_MW',
       'THERM', 'DISP', 'NDISP', 'STOR', 'DR', 'HYDRO', 'Commit', 'RPS', 'CES',
       'New_Build', 'Hydro_level', 'cluster', 'spur_capex', 'spur_inv_mwyr',
       'offshore_spur_capex', 'offshore_spur_inv_mwyr', 'tx_capex',
       'tx_inv_mwyr', 'plant_inv_cost_mwyr', 'R_ID', 'zone', 'voltage_level',
       'CapRes', 'H

#### Spur line/interconnection distances and capacity limits
Interconnection distances and the maximum available capacity for UtilityPV, LandbasedWind, and OffshoreWind are all included in the data used to select and combine clusters. Spur line distances for other resources must be provided by the user in a CSV file and included in the settings file as `capacity_limit_spur_fn`.

In [54]:
cols = [
    "region",
    "technology",
    "cluster",
    "R_ID",
    "Max_Cap_MW",
    "lcoe",
    "regional_cost_multiplier",
    "Inv_cost_per_MWyr",
    "plant_inv_cost_mwyr",
    "interconnect_annuity",
    "spur_inv_mwyr",
    "spur_miles",
    "offshore_spur_inv_mwyr",
    "tx_inv_mwyr",
    "profile",
]
new_gen[cols]

,region,technology,cluster,R_ID,Max_Cap_MW,lcoe,regional_cost_multiplier,Inv_cost_per_MWyr,plant_inv_cost_mwyr,interconnect_annuity,spur_inv_mwyr,spur_miles,offshore_spur_inv_mwyr,tx_inv_mwyr,profile
0,CA_N,NaturalGas_CCCCSAvgCF_Mid,1.0,NaN,-1.00000,0.000000,1.090697,234923.197979,221996.0,0.000000,12927.197979,20.000000,0.000000,0.0,0
1,CA_N,NaturalGas_CCAvgCF_Mid,1.0,NaN,-1.00000,0.000000,1.339623,117344.000000,117344.0,0.000000,0.000000,0.000000,0.000000,0.0,0
2,CA_N,NaturalGas_CTAvgCF_Mid,1.0,NaN,-1.00000,0.000000,1.211268,91163.000000,91163.0,0.000000,0.000000,0.000000,0.000000,0.0,0
3,CA_N,Battery_*_Mid,1.0,NaN,-1.00000,0.000000,1.041215,54124.000000,54124.0,0.000000,0.000000,0.000000,0.000000,0.0,0
4,CA_N,Nuclear_mid__,1.0,NaN,-1.00000,0.000000,1.249179,613037.994947,580720.0,0.000000,32317.994947,50.000000,0.000000,0.0,0
5,CA_N,NaturalGas_CCS100_Mid,1.0,NaN,-1.00000,0.000000,1.090697,247277.197979,234350.0,0.000000,12927.197979,20.000000,0.000000,0.0,0
6,CA_N,LandbasedWind_LTRG1_Mid,1.0,NaN,1259.55000,109.317400,2.109174,326281.493100,288184.0,38097.493100,8045.241662,12.447000,0.000000,0.0,"[0.08843548, 0.22878414, 0.25738907, 0.3396663..."
7,CA_N,LandbasedWind_LTRG1_Mid,1.0,NaN,7164.88140,101.403571,2.109174,324869.661345,288184.0,36685.661345,19797.432734,30.629117,0.000000,0.0,"[0.034569643, 0.03650082, 0.055118807, 0.05934..."
8,CA_N,OffShoreWind_OTRG10_Mid,1.0,NaN,18920.54330,161.759154,1.308483,939767.255090,476591.0,463176.255090,140242.185591,216.972287,177276.430331,0.0,"[0.05657291, 0.028173681, 0.09259045, 0.070540..."
9,CA_N,OffShoreWind_OTRG10_Mid,1.0,NaN,15748.00000,172.627669,1.308483,856025.037885,476591.0,379434.037885,45766.799857,70.806991,229905.779571,0.0,"[0.34054095, 0.4315498, 0.3708048, 0.34306705,..."


#### Generation profiles

Hourly generation profiles are saved in a `variability` column of the dataframe. These are then extracted using the function `make_generator_variability`.

In [70]:
existing_variability = make_generator_variability(existing_gen)
existing_variability

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.0,0.216364,1.0,1.0,1.0,1.0,0.0004,0.216364,0.2899,1.0,0.105270,1.0,1.0,1.0,1.0,0.016339,0.105270,0.282178
1,1.0,0.216121,1.0,1.0,1.0,1.0,0.0293,0.216121,0.0000,1.0,0.105162,1.0,1.0,1.0,1.0,0.112207,0.105162,0.000000
2,1.0,0.215877,1.0,1.0,1.0,1.0,0.0617,0.215877,0.0000,1.0,0.105108,1.0,1.0,1.0,1.0,0.302212,0.105108,0.000000
3,1.0,0.215634,1.0,1.0,1.0,1.0,0.0811,0.215634,0.0000,1.0,0.105053,1.0,1.0,1.0,1.0,0.506039,0.105053,0.000000
4,1.0,0.215384,1.0,1.0,1.0,1.0,0.0819,0.215384,0.0000,1.0,0.104999,1.0,1.0,1.0,1.0,0.476053,0.104999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,0.217673,1.0,1.0,1.0,1.0,0.0204,0.217673,0.6897,1.0,0.105595,1.0,1.0,1.0,1.0,0.094797,0.105595,0.689717
8756,1.0,0.217430,1.0,1.0,1.0,1.0,0.0000,0.217430,0.7108,1.0,0.105541,1.0,1.0,1.0,1.0,0.076850,0.105541,0.731451
8757,1.0,0.217187,1.0,1.0,1.0,1.0,0.0000,0.217187,0.6974,1.0,0.105433,1.0,1.0,1.0,1.0,0.021176,0.105433,0.730564
8758,1.0,0.216943,1.0,1.0,1.0,1.0,0.0000,0.216943,0.6187,1.0,0.105377,1.0,1.0,1.0,1.0,0.003680,0.105377,0.640714


In [71]:
existing_variability.columns = (
    existing_gen["region"]
    + "_"
    + existing_gen["Resource"]
    + "_"
    + existing_gen["cluster"].astype(str)
)
existing_variability

,CA_N_biomass_1,CA_N_conventional_hydroelectric_1,CA_N_geothermal_1,CA_N_hydroelectric_pumped_storage_1,CA_N_natural_gas_fired_combined_cycle_1,CA_N_natural_gas_fired_combustion_turbine_1,CA_N_onshore_wind_turbine_1,CA_N_small_hydroelectric_1,CA_N_solar_photovoltaic_1,CA_S_biomass_1,CA_S_conventional_hydroelectric_1,CA_S_geothermal_1,CA_S_hydroelectric_pumped_storage_1,CA_S_natural_gas_fired_combined_cycle_1,CA_S_natural_gas_fired_combustion_turbine_1,CA_S_onshore_wind_turbine_1,CA_S_small_hydroelectric_1,CA_S_solar_photovoltaic_1
0,1.0,0.216364,1.0,1.0,1.0,1.0,0.0004,0.216364,0.2899,1.0,0.105270,1.0,1.0,1.0,1.0,0.016339,0.105270,0.282178
1,1.0,0.216121,1.0,1.0,1.0,1.0,0.0293,0.216121,0.0000,1.0,0.105162,1.0,1.0,1.0,1.0,0.112207,0.105162,0.000000
2,1.0,0.215877,1.0,1.0,1.0,1.0,0.0617,0.215877,0.0000,1.0,0.105108,1.0,1.0,1.0,1.0,0.302212,0.105108,0.000000
3,1.0,0.215634,1.0,1.0,1.0,1.0,0.0811,0.215634,0.0000,1.0,0.105053,1.0,1.0,1.0,1.0,0.506039,0.105053,0.000000
4,1.0,0.215384,1.0,1.0,1.0,1.0,0.0819,0.215384,0.0000,1.0,0.104999,1.0,1.0,1.0,1.0,0.476053,0.104999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,0.217673,1.0,1.0,1.0,1.0,0.0204,0.217673,0.6897,1.0,0.105595,1.0,1.0,1.0,1.0,0.094797,0.105595,0.689717
8756,1.0,0.217430,1.0,1.0,1.0,1.0,0.0000,0.217430,0.7108,1.0,0.105541,1.0,1.0,1.0,1.0,0.076850,0.105541,0.731451
8757,1.0,0.217187,1.0,1.0,1.0,1.0,0.0000,0.217187,0.6974,1.0,0.105433,1.0,1.0,1.0,1.0,0.021176,0.105433,0.730564
8758,1.0,0.216943,1.0,1.0,1.0,1.0,0.0000,0.216943,0.6187,1.0,0.105377,1.0,1.0,1.0,1.0,0.003680,0.105377,0.640714


In [46]:
make_generator_variability(new_gen)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,1.0,1.0,1.0,1.0,1.0,1.0,0.088435,0.034570,0.056573,0.340541,0.053322,1.0,1.0,1.0,1.0,1.0,1.0,0.064086,0.072826,0.070055,0.045678,0.381206,0.354309,0.392350,0.430983,0.380811,0.089249,0.089300
1,1.0,1.0,1.0,1.0,1.0,1.0,0.228784,0.036501,0.028174,0.431550,0.003695,1.0,1.0,1.0,1.0,1.0,1.0,0.148605,0.118585,0.130442,0.086721,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.083399
2,1.0,1.0,1.0,1.0,1.0,1.0,0.257389,0.055119,0.092590,0.370805,0.042638,1.0,1.0,1.0,1.0,1.0,1.0,0.319481,0.082894,0.179688,0.113493,0.000000,0.000000,0.000000,0.000000,0.000000,0.032116,0.032127
3,1.0,1.0,1.0,1.0,1.0,1.0,0.339666,0.059342,0.070540,0.343067,0.056868,1.0,1.0,1.0,1.0,1.0,1.0,0.474706,0.134995,0.272368,0.274733,0.000000,0.000000,0.000000,0.000000,0.000000,0.024341,0.024390
4,1.0,1.0,1.0,1.0,1.0,1.0,0.410760,0.082231,0.018969,0.323002,0.012820,1.0,1.0,1.0,1.0,1.0,1.0,0.474848,0.210213,0.351928,0.346992,0.000000,0.000000,0.000000,0.000000,0.000000,0.015382,0.015473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,1.0,1.0,1.0,1.0,1.0,0.098973,0.355000,0.007967,0.154682,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.237970,0.029499,0.076308,0.122317,0.906424,0.931416,0.873197,0.940755,0.920576,0.621535,0.621689
8756,1.0,1.0,1.0,1.0,1.0,1.0,0.116479,0.311046,0.004159,0.037722,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.181572,0.045488,0.081166,0.126690,0.930923,0.944681,0.933979,0.957254,0.955774,0.604462,0.604511
8757,1.0,1.0,1.0,1.0,1.0,1.0,0.021648,0.264090,0.000000,0.008297,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.087203,0.010657,0.049913,0.065898,0.952745,0.932476,0.901867,0.945872,0.949151,0.511832,0.511802
8758,1.0,1.0,1.0,1.0,1.0,1.0,0.011410,0.210792,0.000000,0.004214,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.039239,0.000517,0.040333,0.007913,0.857150,0.843580,0.815241,0.850169,0.841343,0.089249,0.089300


In [17]:
make_generator_variability(new_gen, gc.settings)

,0.0_ev_load_shifting,0.0_ev_load_shifting,nan_naturalgas_ccccsavgcf_mid,nan_naturalgas_ccavgcf_mid,nan_naturalgas_ctavgcf_mid,nan_battery_mid,nan_nuclear_mid,nan_naturalgas_ccs100_mid,nan_landbasedwind_ltrg1_mid,nan_landbasedwind_ltrg1_mid,nan_offshorewind_otrg10_mid,nan_offshorewind_otrg10_mid,nan_offshorewind_otrg10_mid,nan_naturalgas_ccccsavgcf_mid,nan_naturalgas_ccavgcf_mid,nan_naturalgas_ctavgcf_mid,nan_battery_mid,nan_nuclear_mid,nan_naturalgas_ccs100_mid,nan_landbasedwind_ltrg1_mid,nan_landbasedwind_ltrg1_mid,nan_landbasedwind_ltrg1_mid,nan_landbasedwind_ltrg1_mid,nan_utilitypv_losangeles_mid,nan_utilitypv_losangeles_mid,nan_utilitypv_losangeles_mid,nan_utilitypv_losangeles_mid,nan_utilitypv_losangeles_mid
1,0.089,0.089,1.0,1.0,1.0,1.0,1.0,1.0,0.088435,0.034570,0.056573,0.340541,0.053322,1.0,1.0,1.0,1.0,1.0,1.0,0.064086,0.072826,0.070055,0.045678,0.381206,0.354309,0.392350,0.430983,0.380811
2,0.083,0.083,1.0,1.0,1.0,1.0,1.0,1.0,0.228784,0.036501,0.028174,0.431550,0.003695,1.0,1.0,1.0,1.0,1.0,1.0,0.148605,0.118585,0.130442,0.086721,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.032,0.032,1.0,1.0,1.0,1.0,1.0,1.0,0.257389,0.055119,0.092590,0.370805,0.042638,1.0,1.0,1.0,1.0,1.0,1.0,0.319481,0.082894,0.179688,0.113493,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.024,0.024,1.0,1.0,1.0,1.0,1.0,1.0,0.339666,0.059342,0.070540,0.343067,0.056868,1.0,1.0,1.0,1.0,1.0,1.0,0.474706,0.134995,0.272368,0.274733,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.015,0.015,1.0,1.0,1.0,1.0,1.0,1.0,0.410760,0.082231,0.018969,0.323002,0.012820,1.0,1.0,1.0,1.0,1.0,1.0,0.474848,0.210213,0.351928,0.346992,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,0.622,0.622,1.0,1.0,1.0,1.0,1.0,1.0,0.098973,0.355000,0.007967,0.154682,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.237970,0.029499,0.076308,0.122317,0.906424,0.931416,0.873197,0.940755,0.920576
8757,0.604,0.605,1.0,1.0,1.0,1.0,1.0,1.0,0.116479,0.311046,0.004159,0.037722,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.181572,0.045488,0.081166,0.126690,0.930923,0.944681,0.933979,0.957254,0.955774
8758,0.512,0.512,1.0,1.0,1.0,1.0,1.0,1.0,0.021648,0.264090,0.000000,0.008297,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.087203,0.010657,0.049913,0.065898,0.952745,0.932476,0.901867,0.945872,0.949151
8759,0.089,0.089,1.0,1.0,1.0,1.0,1.0,1.0,0.011410,0.210792,0.000000,0.004214,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.039239,0.000517,0.040333,0.007913,0.857150,0.843580,0.815241,0.850169,0.841343


In [22]:
gc.settings["demand_response_resources"]

{2030: {'ev_load_shifting': {'fraction_shiftable': 0.8,
   'parameter_values': {'Max_DSM_delay': 5, 'DR': 2}}},
 2045: {'ev_load_shifting': {'fraction_shiftable': 0.8,
   'parameter_values': {'Max_DSM_delay': 5, 'DR': 2}}}}

In [30]:
dr_profile = gc.demand_response_profiles["ev_load_shifting"]
dr_cf = dr_profile / dr_profile.max()
dr_cf

,CA_N,CA_S
0,0.089249,0.089300
1,0.083333,0.083399
2,0.032116,0.032127
3,0.024341,0.024390
4,0.015382,0.015473
...,...,...
8755,0.621535,0.621689
8756,0.604462,0.604511
8757,0.511832,0.511802
8758,0.089249,0.089300


In [39]:
list(dr_cf.values.T)

[array([0.08924949, 0.08333333, 0.03211629, ..., 0.51183232, 0.08924949,
        0.08924949]),
 array([0.08929976, 0.0833989 , 0.03212693, ..., 0.51180173, 0.08929976,
        0.08929976])]

In [43]:
list(dr_cf.values.T)

[array([0.08924949, 0.08333333, 0.03211629, ..., 0.51183232, 0.08924949,
        0.08924949]),
 array([0.08929976, 0.0833989 , 0.03212693, ..., 0.51180173, 0.08929976,
        0.08929976])]

In [22]:
dr_path = gc.settings["input_folder"] / gc.settings["demand_response_fn"]
resources = gc.settings["demand_response_resources"][gc.settings["model_year"]].keys()
for resource in resources:
    dr_profiles = make_demand_response_profiles(dr_path, resource, gc.settings)
    dr_profiles = dr_profiles / dr_profiles.max()

    assert dr_profiles.max().max() == 1

In [23]:
print(dr_profiles)

          CA_N      CA_S
0     0.089249  0.089300
1     0.083333  0.083399
2     0.032116  0.032127
3     0.024341  0.024390
4     0.015382  0.015473
...        ...       ...
8755  0.621535  0.621689
8756  0.604462  0.604511
8757  0.511832  0.511802
8758  0.089249  0.089300
8759  0.089249  0.089300

[8760 rows x 2 columns]


In [36]:
model_gdf = load_ipm_shapefile(gc.settings)

In [37]:
model_gdf

,IPM_Region,geometry,model_region
48,WEC_BANC,"MULTIPOLYGON (((-137062.964 -20577.766, -13352...",CA_N
49,WEC_CALN,"MULTIPOLYGON (((-26905.041 -343681.519, -26858...",CA_N
50,WEC_LADW,"MULTIPOLYGON (((147531.138 -578110.363, 147523...",CA_S
51,WEC_SDGE,"MULTIPOLYGON (((261508.595 -578272.714, 261412...",CA_S
55,WECC_IID,"POLYGON ((499663.896 -494815.927, 497706.026 -...",CA_S
60,WECC_SCE,"MULTIPOLYGON (((43384.114 -525298.005, 50284.5...",CA_S


In [38]:
model_gdf.dissolve(by="model_region")

,geometry,IPM_Region
model_region,,
CA_N,"MULTIPOLYGON (((-264550.059 -30640.048, -26457...",WEC_BANC
CA_S,"MULTIPOLYGON (((499663.896 -494815.927, 497706...",WEC_LADW
